# Load sklearn model from pickle file
also load data set

In [20]:
import pickle
model = pickle.load(open("../models/svm_landmark_model2.sav", "rb"))

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
dataFrame = pd.read_csv("../asl_alphabet_train/asl_alphabet_landmarks.csv")
labels = dataFrame.iloc[:, -1].values
inputs = np.reshape(dataFrame.iloc[:, 1 + 3:-1].values, (-1, 60))

train_set, test_set, train_labels, test_labels = train_test_split(inputs, labels, test_size=0.01, random_state=0)

# Convert or export the model into ONNX format

In [21]:
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType

initial_type = [('float_input', FloatTensorType([None, train_set.shape[1]]))]
onx = convert_sklearn(model, initial_types=initial_type)
with open("./svm_landmarks2.onnx", "wb") as f:
    f.write(onx.SerializeToString())

# Load and run the model using ONNX Runtime

In [22]:
import onnxruntime as rt
sess = rt.InferenceSession("./svm_landmarks2.onnx", providers=["CPUExecutionProvider"])
input_name = sess.get_inputs()[0].name

pred_onx = sess.run([], {input_name: test_set.astype(np.float32)})

In [23]:
print(pred_onx[1])

[{0: 3.9434868085663766e-05, 1: 2.1025069145252928e-05, 2: 2.6400579372420907e-05, 3: 2.9639135391335003e-05, 4: 9.92912900983356e-05, 5: 5.360359136830084e-05, 6: 4.382024144433672e-06, 7: 4.220396931486903e-06, 8: 3.385534364497289e-05, 9: 4.892338893114356e-06, 10: 0.0005131217185407877, 11: 0.00011440993694122881, 12: 2.7189094907953404e-05, 13: 8.420773156103678e-06, 14: 3.628234480856918e-05, 15: 8.030966455407906e-06, 16: 7.562147857242962e-06, 17: 0.0016809868393465877, 18: 1.779606827767566e-05, 19: 0.00010662681597750634, 20: 0.9909040927886963, 21: 0.003352069528773427, 22: 0.0024183238856494427, 23: 0.00019184443226549774, 24: 1.2904137292935047e-05, 25: 0.00027319835498929024, 26: 4.845815965381917e-06, 27: 5.46737692275201e-06}, {0: 6.47174019832164e-05, 1: 7.446091331075877e-05, 2: 1.0794711670314427e-05, 3: 1.584589517733548e-05, 4: 0.9957298636436462, 5: 1.3749731806456111e-05, 6: 1.1338453987264074e-05, 7: 8.659790182719007e-06, 8: 0.0015809317119419575, 9: 2.00195081

In [24]:
print(test_labels)

[20  4 18 11 13  0 17  0 26  9 24  9 10  8 23  7 22  1  8 23 15 11  8 15
 11  8  5  7 11 22  6 17 23  9  8 10 19 20 22  4 20  6  6 20  1 21  8 22
 23  8 11 22 27  5 10 21 10  4 19 22 18 15 22 23 27 11  5  1  6 24  4 21
  8 24  2 19  3  7  4  2 18 15 18  7 27 22 12 22 17 16 20 20 16  5  4 12
 19  4 22  8 24  5 17  7  0 27 22 25 15  4  5 11 15 19 16 16  5  7 22  5
 20 12  0 19 12 19 10 21  7 17 17 23 17 25  1 12  9 27 20 25  2 14 11  1
  0  9 24  3  5 19 23  8  0 12 16  7 11 18 10 21 26  3  1  5 17 10 22 13
  7 13  6 12 11 11 23  6 12  4  5 17  4  6 21  5 23  4  1  9  4  2 16  8
 19 27 26  8  8 17  5 14 10 12  0 16 10 11 19 27  9  6  9 15 22 27  9 11
  9  8 27 22 19 25  3 14  7  9  4 26 10  0  0  7  8  3  8 20  6  1 26 17
 14  9  0 25 26 14  8 25  1  3  2 17 24 26 12 16  5 26 24  0 16  6  2  6
  0  0 14  8  0 11 23 24 10 22 20 27 23 22 22  9  5  6 11 17 12 16 10  3
 15  6  4  0  9  1 13  7 14  9 21  9  9 21  4 17 21 21 26  0 24  4  7 25
  3 13  2 25  7  9  1  8  7 12 14 19  1  7  6  7  4

In [25]:
print(sum(pred_onx[0] == test_labels))
print(len(test_labels))
print(sum(pred_onx[0] == test_labels)/len(test_labels))

628
637
0.9858712715855573
